# 5. How To Read And Write To A Google Sheet With Python & Pandas

## Learning Outcomes

- To learn how to authenticate with Google Sheets within a .py file or jupyter notebook file.
- To learn how to read from a Google Sheet.
- To learn how to write (upload) to a Google Sheet.
- To learn how to maniuplate data within Google Sheets.
- To learn how to store data to a local file from a Google Sheet.

---------------------------------------------------------------

## Module Imports

In order to run the module imports below you will need to have installed the following python client:

- https://github.com/googleapis/google-api-python-client

A quick way to do that would be to run the following command and to restart your Jupyter Notebook server:

~~~

pip install --upgrade google-api-python-client

~~~

---

Additionally I would encourage you to download the gcloud software development kit (SDK), which will come in useful for deploying and access your Google Cloud Platform resources:

You can find more information on how to do that from:

- https://cloud.google.com/python/setup
- https://cloud.google.com/sdk/docs

In [8]:
pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /opt/anaconda3/lib/python3.7/site-packages (1.8.4)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import json
import csv
from google.oauth2 import service_account

----------------------------------------------------------------

## Download Pygsheets

We will be using a python package called [pygsheets](https://pygsheets.readthedocs.io/en/stable/) to easily interact with a new google sheet that we will create.

If you're using anaconda you can navigate to your terminal and install the latest version of this package with:

~~~

conda install pygsheets

~~~

Alternatively if you're using pip or pip3 to manage your packages you can type either:

~~~

pip install pygsheets
pip3 install pygsheets


~~~

In [10]:
!pip install pygsheets

As you can see from above I've previously installed pygsheets, after you've installed the python package if you're working from a jupyter notebook, then please restart the Kernel:

![how to clear the kernel](https://sempioneer.com/wp-content/uploads/2020/05/1_clear_kernel.png)

In [11]:
import pygsheets

--------------------------------------------------------------------------------------------------------------------------------

## To Authenticate To A Google Cloud Project With A .JSON Service Account Key

Using your local .json file we will authenticate to our google service account.

In [12]:
with open('service_account.json') as source:
    info = json.load(source)
    
credentials = service_account.Credentials.from_service_account_info(info)

--------------------------------------------------------

## Create A New Google Sheet + Obtain The Unique ID

Firstly we are going to create a new google sheet, then we'll obtain the id of that specific google sheet which can be found within the URL:

![how to obtain the google sheet id](https://sempioneer.com/wp-content/uploads/2020/05/2_obtain_the_google_sheet_id.png)

## Python With Google Sheet Commands

### Authenticating With Google Sheets With Pyghseets

Let's see how to successfully authenticate to google sheets with our .json key 

In [17]:
client = pygsheets.authorize(service_account_file='service_account.json')

---------------------------------------------------------------------------------

### How to connect to a specific google sheet

Now that we've authenticated pygsheets with our google cloud project let's connect to a specific google sheet. In order to do this we will need to grab the URL link and share the spreadsheet.

In [ ]:
spreadsheet_url = \
"https://docs.google.com/spreadsheets/d/1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q/edit?usp=sharing"

Additionally we will extract the spreadsheet ID which can be located in the URL above between /d/ and /edit?

This id will be unique for you so please make sure to code along! 

![google spreadsheet](https://sempioneer.com/wp-content/uploads/2020/05/spreadsheet-id-.png)

In [19]:
sheet_data = client.sheet.get('1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q')

There are three ways to open the spreadsheet:

- By the spreadsheet title
- By the unique spreadsheet ID
- By the exact URL of the ghseet

Let's showcase all three and you can take your pick!

In [26]:
# sheet = client.open('Sheet1') ## You will need to activate the Google Drive API as well as the spreadsheet API for this one to work
sheet = client.open_by_key('1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q')
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OBR8HEE2rjaWqZPnb8mHnp3YpxxELcy5Zqve3gtX22Q/edit?usp=sharing')

In [28]:
print(sheet)

<Spreadsheet 'Test Google Sheet' Sheets:1>


------------------------------------------------------------------------------------------

### How To Select A Specific Google Worksheet

Let's select a worksheet was automatically created with the gsheet called Sheet1:

In [32]:
wks = sheet.worksheet_by_title('Sheet1')
print(wks)

<Worksheet 'Sheet1' index:0>


-----------------------------------------------------------------

### How To Upload Data To A Google Sheet From A Pandas DataFrame

I am going to use the keyword data that I originally downloaded from Ahrefs, however if you have another csv that's absolutely fine. I'd encourage you to simply read any csv file and to create a pandas dataframe:

In [50]:
df = pd.read_csv('../1_Keywords/data/digital-marketing-keyword-ideas.csv', 
                 delimiter='\t', 
                 encoding='UTF-16')

df.drop(columns=['#'], inplace=True)

To upload a dataframe to this worksheet we can use the following syntax:
    
~~~

set_dataframe(df, start, copy_index=False, copy_head=True, extend=False, fit=False, escape_formulae=False, **kwargs)

~~~

In [101]:
wks.set_dataframe(df, start=(1,1))

![](https://sempioneer.com/wp-content/uploads/2020/05/python-pygsheets.png)

It's worth remembering that when you run the above command you will overwrite any cell values. Instead if you would like to progressively append rows within your google sheets please use the below syntax:

~~~
wks.set_dataframe(df, start=(1,1), extend=True)

~~~

--------------------------------------------------------------------

### How To A Pandas Dataframe From A Specific Worksheet

We can also get data directly from the google sheets worksheet in the form of a dataframe:

~~~

get_as_df(has_header=True, index_column=None, start=None, end=None, numerize=True, empty_value='', value_render=<ValueRenderOption.FORMATTED_VALUE: 'FORMATTED_VALUE'>, **kwargs)[source]

~~~

In [54]:
dataframe_two = wks.get_as_df()

In [56]:
print(f"This new dataframe has {dataframe_two.shape[0]} rows and {dataframe_two.shape[1]} columns")

This new dataframe has 1000 rows and 11 columns


------------------------------------------------------------------------------------------

### How To Get All Of The Google Sheet Values In A Python Format

We can also get all of the values in a list of lists:

In [59]:
all_values = wks.get_all_values()

In my case, notice how this has picked up all of the empty spaces that are located on the right hand side of the worksheet:

In [61]:
print(all_values[0])

['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks', 'CPS', 'Return Rate', 'Parent Keyword', 'Last Update', 'SERP Features', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


A way to quickly remove the empty strings per list would be do a <strong> nested list comprehension: </strong>

In [64]:
cleaned_values = [[item for item in unique_list if item ]for unique_list in all_values]

------------------------------------------------

### How To Get Cell Ranges In Google Sheets With Python

You can also specific ranges similar to your excel functions for getting custom columns and rows.

In [87]:
cell_range = wks.range('A1:F10', 
          returnas='matrix')

In [90]:
print(cell_range)

[['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks'], ['coast', 'gb', '42', '70000', '2.5', '64715'], ['hubspot', 'gb', '67', '63000', '5', '59708'], ['digital marketing', 'gb', '74', '19000', '7', '11033'], ['digital', 'gb', '89', '16000', '2.5', '5912'], ['content meaning', 'gb', '45', '4400', '17', '622'], ['digital media', 'gb', '24', '3600', '3', '1671'], ['digital marketing agency', 'gb', '57', '3400', '18', '3925'], ['digital uk', 'gb', '24', '3100', '1.2', '2402'], ['what is digital marketing', 'gb', '74', '3100', '2.5', '2119']]


-----------------------------------------------------------------------------

### How To Get A Single Row For Extracting The Column Headers

We can get a single row with 

~~~ 

wks.get_row(row, returnas='matrix', include_tailing_empty=True, **kwargs)

~~~

In [78]:
headers = wks.get_row(1, include_tailing_empty=False)

In [81]:
print(headers)

['Keyword', 'Country', 'Difficulty', 'Volume', 'CPC', 'Clicks', 'CPS', 'Return Rate', 'Parent Keyword', 'Last Update', 'SERP Features']


Also notice how we've changed the include_tailing_empty to False which has automatically removed any empty strings in that row.

------------------------------------------------------------------------------------

### How To Extract A Single Column

Sometimes you might want to select a specific column from your worksheet:

In [67]:
first_column = wks.get_col(1)
first_column_data = first_column[1:] # We are doing a slide here to remove the header (keyword)

------------------------------------------------------------------------------------

### How To Sort By A Column

You can easily sort the google sheet by a specific column with:
    
~~~

wks.sort_range(start, end, basecolumnindex=0, sortorder='ASCENDING')
# Sorts the data in rows based on the given column index.


~~~

In [106]:
wks.sort_range(start='A2', end='L1001',basecolumnindex=4, sortorder='DESCENDING' )

<strong> Its worth knowing that np.nans (not a number) </strong> are treated as large numbers within google sheets. Therefore let's clear the sheet, remove all of the nans before uploading the dataframe, then we will perform this operation again.

![](https://sempioneer.com/wp-content/uploads/2020/05/python-pygsheets-nans.png)

------------------------------------------------------------------------------------

### How To Clear A Google Sheet With Python

You can clear all of the existing rows and columns within the sheet by referencing the same workspace variable (wks) and the .clear() syntax: 

~~~

wks.clear()

~~~

In [107]:
wks.clear()

----------------------------------------------------------------------

Now I'd recommend removing any nans before uploading your dataframe! So let's remove any np.nans within the Volume Column:

In [111]:
df.dropna(subset=['Volume'], inplace=True)
wks.set_dataframe(df, start=(1,1))

Let's try sorting again by the Volume column with pygsheets:

In [113]:
wks.sort_range(start='A2', end='L1001',basecolumnindex=4, sortorder='DESCENDING' )

![](https://sempioneer.com/wp-content/uploads/2020/05/sorted_volume.png)

-----------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------

In [ ]:
How To Insert Multiple Columns

In [ ]:
How To Insert A Column

In [ ]:
How To Insert A Row

---------------------------------------------------------------

### How To Delete A Single Row

------------------------------------------------------

### How To Delete A Single Column

-------------------------------------------------------------------------------------

### How To Export A Google Sheet To A .CSV

Also its possible to easily export your google sheet to a .csv file with: 

~~~

wks.export(file_format=<ExportType.CSV: 'text/csv:.csv'>, filename=None, path='')

~~~

However <strong> we'll need to enable the Google Drive API </strong> for this to work, so quickly go to your Google Cloud Project and activate this API for the following command to work:

![](https://sempioneer.com/wp-content/uploads/2020/05/googel-drive-api.png)

In [117]:
wks.export(filename='this_is_a_csv_file')

-------------------------------------------------------------------------------------

---------------------------------------------

In [ ]:
- How get_range
- How to resize a sheet
- How to pull data from multiple worksheets
- How to create a worksheet
- How to delete a worksheet
- How to share a worksheet